In [ ]:
import pandas as pd
import json
import os
from collections import Counter
import matplotlib.pyplot as plt
import re
from datetime import datetime
import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression

# Function

In [ ]:
def format_number(num):
    return f"{num:,.0f}".replace(",", ".")

## Aired `random date` to `month-year`

In [ ]:
def extract_first_release(aired_str):
    """
    Extract first month and year from aired string
    
    Parameters:
    aired_str (str): The aired date string to process
    
    Returns:
    str: Release date in MM-YYYY format or None if cannot be processed
    """
    if pd.isna(aired_str) or aired_str == '':
        return None
    
    # Dictionary untuk konversi nama bulan ke angka
    month_dict = {
        'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04',
        'May': '05', 'Jun': '06', 'Jul': '07', 'Aug': '08',
        'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'
    }
    
    # Bersihkan string dari dash di awal
    aired_str = aired_str.strip('-')
    
    try:
        # Case 1: Format dengan bulan dan tahun (e.g., "Dec-1,-1933" atau "Aug-29,-1965-to-Jun-28,-1967")
        month_year_pattern = r'([A-Za-z]+)-\d+,?-?(\d{4})'
        match = re.search(month_year_pattern, aired_str)
        if match:
            month_str, year = match.groups()
            month_num = month_dict.get(month_str[:3])
            if month_num:
                return f"{month_num}-{year}"
        
        # Case 2: Format tahun saja (e.g., "2017")
        year_pattern = r'(\d{4})'
        match = re.search(year_pattern, aired_str)
        if match:
            year = match.group(1)
            return f"01-{year}"  # Default ke Januari untuk tahun saja
        
        # Case 3: Format bulan dan tahun tanpa tanggal (e.g., "Apr-2001")
        month_year_simple = r'([A-Za-z]+)-(\d{4})'
        match = re.search(month_year_simple, aired_str)
        if match:
            month_str, year = match.groups()
            month_num = month_dict.get(month_str[:3])
            if month_num:
                return f"{month_num}-{year}"
                
        return None
        
    except (ValueError, AttributeError):
        return None

## Analysis By Genre

In [ ]:
def genre_analysis(genre_analysis, dataframe, limit=0):
    df = dataframe
    df_analysis = df[df['genre'].apply(lambda x: genre_analysis in x)].copy()  # Use .copy() to avoid chained assignment
    df_analysis.loc[:, 'aired'] = pd.to_datetime(df_analysis['aired'], format='%m-%Y')  # Use .loc
    df_analysis.loc[:, 'year'] = df_analysis['aired'].dt.year  # Use .loc
    df_analysis.loc[:, 'month'] = df_analysis['aired'].dt.month  # Use .loc

    # Get the current year
    current_year = datetime.now().year

    # Filter data based on the limit (number of years back)
    if limit > 0:
        df_analysis = df_analysis[df_analysis['year'] >= (current_year - limit)]

    monthly_stats = df_analysis.groupby(['year', 'month']).size().reset_index(name='count')
    monthly_stats['date'] = pd.to_datetime(monthly_stats[['year', 'month']].assign(day=1))

    plt.figure(figsize=(20, 10))
    plt.plot(monthly_stats['date'], monthly_stats['count'], marker='o', linestyle='-', color='blue')
    plt.title(f'Monthly Statistics of {genre_analysis} Aired Data', fontsize=24)
    plt.xlabel('Date', fontsize=24)
    plt.ylabel('Count', fontsize=24)
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    # print(monthly_stats.to_markdown())
    return monthly_stats

## Genre per Year

In [ ]:
def genre_analysis_decade(genre_analysis, dataframe, limit=0):
    df = dataframe
    
    df_analysis = df[df['genre'].apply(lambda x: genre_analysis in x)].copy()  # Use .copy() to avoid chained assignment
    
    df_analysis.loc[:, 'aired'] = pd.to_datetime(df_analysis['aired'], format='%m-%Y')  # Use .loc
    df_analysis.loc[:, 'year'] = df_analysis['aired'].dt.year  # Use .loc
    df_analysis.loc[:, 'month'] = df_analysis['aired'].dt.month  # Use .loc
    
    current_year = datetime.now().year
        
    df_analysis['decade'] = (df_analysis['year'] // limit) * limit
    decade_stats = df_analysis.groupby('decade').size().reset_index(name='count')
    
    # Plot the data
    plt.figure(figsize=(12, 6))
    plt.bar(decade_stats['decade'], decade_stats['count'], color='blue')
    plt.title(f'Decade Statistics of {genre_analysis} Aired Data (Last {current_year} Years)', fontsize=24)
    plt.xlabel('Decade', fontsize=24)
    plt.ylabel('Count', fontsize=24)
    plt.grid(True)
    plt.xticks(decade_stats['decade'], rotation=45)
    plt.tight_layout()
    plt.show()
    
    # print(decade_stats.to_markdown())
    
    return decade_stats

## Heatmap Trend

In [ ]:
import seaborn as sns

def plot_genre_heatmap(dataframe, top_n_genres=15, start_year=2000, genres=[]):
    # Process data
    df = dataframe.copy()
    df['aired'] = pd.to_datetime(df['aired'], format='%m-%Y')
    df['year'] = df['aired'].dt.year
    
    # Filter by start year
    df = df[df['year'] >= start_year]
    
    # Count genres by year
    genre_counts = {}
    years = sorted(df['year'].unique())
    
    for year in years:
        year_data = df[df['year'] == year]
        year_genres = {}
        
        for _, row in year_data.iterrows():
            for genre in row['genre']:
                year_genres[genre] = year_genres.get(genre, 0) + 1
        
        genre_counts[year] = year_genres
    
    # Find top N genres across all years
    all_genre_counts = {}
    for year_data in genre_counts.values():
        for genre, count in year_data.items():
            all_genre_counts[genre] = all_genre_counts.get(genre, 0) + count
    
    top_genres = sorted(all_genre_counts.items(), key=lambda x: x[1], reverse=True)[:top_n_genres]
    
    if len(genres) > 0:
        top_genres = [(g, all_genre_counts[g]) for g in genres if g in all_genre_counts]
        
    top_genre_names = [g[0] for g in top_genres]
    
    # Create DataFrame for heatmap
    heatmap_data = []
    for year in years:
        row = {'year': year}
        year_data = genre_counts[year]
        
        for genre in top_genre_names:
            row[genre] = year_data.get(genre, 0)
        
        heatmap_data.append(row)
    
    heatmap_df = pd.DataFrame(heatmap_data)
    heatmap_df.set_index('year', inplace=True)
    
    # Create heatmap
    plt.figure(figsize=(14, 10))
    sns.heatmap(heatmap_df, annot=True, fmt='d', cmap='YlGnBu')
    plt.title('Popular Anime Genres by Year', fontsize=16)
    plt.ylabel('Year', fontsize=12)
    plt.xlabel('Genre', fontsize=12)
    plt.tight_layout()
    plt.show()
    
    return heatmap_df

## Trend Calculation

In [ ]:
# Function to analyze and predict genre trends
def analyze_genre_trends(df, prediction_year=2025, focus_years=5):
    """
    Analyze genre trends and predict popularity for a specific year
    
    Parameters:
    df: DataFrame containing genre counts by year
    prediction_year: Year to predict (default: 2025)
    focus_years: Number of recent years to use for trend analysis (default: 5)
    
    Returns:
    DataFrame with predicted values and growth metrics
    """
    # Create a copy to avoid modifying the original
    trend_df = df.copy()
    
    # Filter for complete years (exclude prediction_year which may be incomplete)
    complete_years = [year for year in trend_df.index if year < prediction_year]
    
    # Get the most recent complete years for trend analysis
    recent_years = complete_years[-focus_years:]
    
    # Prepare results dataframe
    results = pd.DataFrame(index=trend_df.columns)
    
    # Calculate average counts over recent years
    results['recent_avg'] = trend_df.loc[recent_years].mean()
    
    # Calculate growth rate using linear regression
    X = np.array(recent_years).reshape(-1, 1)
    
    growth_rates = []
    predictions = []
    r2_scores = []
    
    for genre in trend_df.columns:
        y = trend_df.loc[recent_years, genre].values
        
        # Fit linear regression model
        model = LinearRegression()
        model.fit(X, y)
        
        # Calculate growth rate (slope)
        growth_rate = model.coef_[0]
        growth_rates.append(growth_rate)
        
        # Make prediction for the target year
        prediction = model.predict([[prediction_year]])[0]
        predictions.append(max(0, prediction))  # Ensure non-negative prediction
        
        # Calculate R-squared to assess trend reliability
        r2 = model.score(X, y)
        r2_scores.append(r2)
    
    results['growth_rate'] = growth_rates
    results['prediction'] = predictions
    results['trend_reliability'] = r2_scores
    
    # Calculate normalized score to rank genres (combining current popularity and growth)
    results['normalized_score'] = (
        (results['recent_avg'] / results['recent_avg'].max()) * 0.6 +
        (results['growth_rate'] / results['growth_rate'].max() if results['growth_rate'].max() > 0 else 0) * 0.4
    )
    
    # Sort by normalized score (descending)
    results = results.sort_values('normalized_score', ascending=False)
    
    return results

In [ ]:
# Function to visualize the results
def visualize_genre_trends(df, analysis_df, top_n=10):
    """
    Visualize genre trends and predictions
    
    Parameters:
    df: Original data DataFrame
    analysis_df: Analysis results DataFrame
    top_n: Number of top genres to display (default: 10)
    """
    top_genres = analysis_df.index[:top_n].tolist()
    
    # Plot historical data and predictions for top genres
    plt.figure(figsize=(14, 8))
    
    for genre in top_genres:
        plt.plot(df.index[-6:-1], df.loc[df.index[-6:-1], genre], marker='o', label=genre)
        
        # Add prediction point
        prediction = analysis_df.loc[genre, 'prediction']
        plt.plot(df.index[-1], prediction, marker='*', markersize=10, 
                 color=plt.gca().lines[-1].get_color())
    
    plt.title('Top Anime Genre Trends and Predictions (2020-2025)', fontsize=16)
    plt.xlabel('Year', fontsize=12)
    plt.ylabel('Number of Anime Titles', fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.legend(title='Genres', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    
    # Plot normalized scores for top genres
    plt.figure(figsize=(12, 8))
    bars = plt.barh(top_genres[::-1], analysis_df.loc[top_genres[::-1], 'normalized_score'], color='lime')
    
    # Add growth rate indicators
    for i, genre in enumerate(top_genres[::-1]):
        growth = analysis_df.loc[genre, 'growth_rate']
        color = 'green' if growth > 0 else 'red'
        plt.text(0.01, i, f"{'↑' if growth > 0 else '↓'}", color=color, 
                 fontsize=14, verticalalignment='center')
    
    plt.title('Predicted Popular Anime Genres for 2025', fontsize=16)
    plt.xlabel('Normalized Popularity Score', fontsize=12)
    plt.tight_layout()
    
    # Create a prediction summary table
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.axis('tight')
    ax.axis('off')
    
    table_data = []
    headers = ['Genre', 'Recent Average', 'Growth Rate', '2025 Prediction', 'Trend Reliability']
    
    for genre in top_genres:
        table_data.append([
            genre,
            f"{analysis_df.loc[genre, 'recent_avg']:.1f}",
            f"{analysis_df.loc[genre, 'growth_rate']:.1f}",
            f"{analysis_df.loc[genre, 'prediction']:.1f}",
            f"{analysis_df.loc[genre, 'trend_reliability']:.2f}"
        ])
    
    table = ax.table(cellText=table_data, colLabels=headers, loc='center', cellLoc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 1.5)
    
    plt.title('Anime Genre Prediction Details for 2025', fontsize=16)
    plt.tight_layout()


In [ ]:
# Function to provide textual analysis of the trends
def generate_trend_analysis(analysis_df, top_n=100):
    """
    Generate a textual analysis of anime genre trends
    
    Parameters:
    analysis_df: Analysis results DataFrame
    top_n: Number of top genres to analyze (default: 10)
    
    Returns:
    String containing the analysis
    """
    top_genres = analysis_df.index[:top_n].tolist()
    
    analysis = "# Anime Genre Trend Analysis for 2025\n\n"
    
    # Overall trend summary
    analysis += "## Overall Trend Summary\n"
    analysis += "Based on data from the past 5 years, the following genres are projected to be most popular in 2025:\n\n"
    
    for i, genre in enumerate(top_genres, 1):
        recent_avg = analysis_df.loc[genre, 'recent_avg']
        growth = analysis_df.loc[genre, 'growth_rate']
        prediction = analysis_df.loc[genre, 'prediction']
        
        trend_direction = "increasing" if growth > 0 else "decreasing"
        growth_percent = abs(growth) / recent_avg * 100 if recent_avg > 0 else 0
        
        analysis += f"{i}. **{genre}**: Currently averaging {recent_avg:.1f} titles per year, "
        analysis += f"with a {trend_direction} trend ({growth_percent:.1f}% per year). "
        analysis += f"Projected to have {prediction:.1f} titles in 2025.\n"
    
    # Rising stars
    rising_genres = analysis_df[analysis_df['growth_rate'] > 10].index.tolist()[:5]
    if rising_genres:
        analysis += "\n## Rising Star Genres\n"
        analysis += "These genres show significant positive growth and may become more prominent:\n\n"
        for genre in rising_genres:
            growth = analysis_df.loc[genre, 'growth_rate']
            analysis += f"- **{genre}**: Growing at {growth:.1f} titles per year\n"
    
    # Conclusion
    analysis += "\n## Conclusion\n"
    analysis += "The anime industry in 2025 is likely to be dominated by "
    analysis += ", ".join([f"**{genre}**" for genre in top_genres[:3]])
    analysis += f", with **{top_genres[0]}** leading the way. "
    analysis += "Cross-genre titles that combine elements from multiple popular categories "
    analysis += "are likely to be particularly successful.\n\n"
    analysis += "The 2025 data is currently incomplete as we're still in early 2025, "
    analysis += "but the trends from previous years provide a reliable indication of what to expect."
    
    return analysis


# Get Data

In [ ]:
file_path = './dataset/data.json'

if os.stat(file_path).st_size == 0:  # Check if the file is empty
    print("Error: JSON file is empty.")
    data = {}  # Set default empty dictionary
else:
    try:
        with open(file_path, "r") as file:
            data = json.load(file)
    except json.JSONDecodeError:
        print("Error: JSON file is corrupt or not formatted correctly.")
        data = {}  # Set default empty dictionary

# Dataset

In [ ]:
num_ = 'all'
# num_ = 10000

df = pd.DataFrame(data)

if num_ != 'all':
    df = df[:num_]
    
df['rank'] = pd.to_numeric(df['rank'], errors='coerce')
df['rank'] = df['rank'].fillna(pd.Series(df.index + 1))

df['mal_id'] = pd.to_numeric(df['mal_id'], errors='coerce').fillna(0)
df['aired'] = df['aired'].apply(extract_first_release)

df = df.sort_values(by='rank')

df['member'] = pd.to_numeric(df['member'].str.replace(',', '').replace('', '0'), errors='coerce').fillna(0).astype(int)
df['favorite'] = pd.to_numeric(df['favorite'].str.replace(',', '').replace('', '0'), errors='coerce').fillna(0).astype(int)

df.to_json('./dataset/dataset.json', orient='records')
df.head(2)

In [ ]:
df.loc[:, 'year'] = pd.to_datetime(df['aired'], format='%m-%Y', errors='coerce').dt.year.astype('Int64')

df['genre'] = df['genre'].apply(lambda x: eval(x) if isinstance(x, str) else x)

df_exploded = df.explode('genre')

genre_year_counts = df_exploded.groupby(['genre', 'year']).size().unstack(fill_value=0)

genre_year_counts = genre_year_counts.reindex(sorted(genre_year_counts.columns), axis=1)

genre_year_counts.to_csv('./dataset/genre_year_counts.csv')

In [ ]:
df['studio'] = df['studio'].apply(lambda x: eval(x) if isinstance(x, str) else x)

df_exploded = df.explode('studio')

studio_year_counts = df_exploded.groupby(['studio', 'year']).size().unstack(fill_value=0)

studio_year_counts = studio_year_counts.reindex(sorted(studio_year_counts.columns), axis=1)

studio_year_counts.to_csv('./dataset/studio_year_counts.csv')

In [ ]:
df['demographic'] = df['demographic'].apply(lambda x: eval(x) if isinstance(x, str) else x)

df_exploded = df.explode('demographic')

demographic_year_counts = df_exploded.groupby(['demographic', 'year']).size().unstack(fill_value=0)

demographic_year_counts = demographic_year_counts.reindex(sorted(demographic_year_counts.columns), axis=1)

demographic_year_counts.to_csv('./dataset/demographic_year_counts.csv')

# Analysis Count per Genres

In [ ]:
df['genre'] = df['genre'].apply(lambda x: eval(x) if isinstance(x, str) else x)

genre_stats = {}

for _, row in df.iterrows():
    genres = row['genre']
    members = row['member']
    favorites = row['favorite']
    
    
    if isinstance(genres, list):
        for genre in genres:
            if genre in genre_stats:
                genre_stats[genre]['count'] += 1
                genre_stats[genre]['total_member'] += members
                genre_stats[genre]['total_favorite'] += favorites
                
                genre_stats[genre]['min_member'] = min(genre_stats[genre]['min_member'], members)
                genre_stats[genre]['max_member'] = max(genre_stats[genre]['max_member'], members)
                genre_stats[genre]['min_favorite'] = min(genre_stats[genre]['min_favorite'], favorites)
                genre_stats[genre]['max_favorite'] = max(genre_stats[genre]['max_favorite'], favorites)
            else:
                genre_stats[genre] = {
                    'count': 1, 
                    'total_member': members,
                    'total_favorite': favorites,
                    'min_member': members,
                    'max_member': members,
                    'min_favorite': favorites,
                    'max_favorite': favorites,
                }

# Konversi dictionary ke DataFrame
df_genre_stats = pd.DataFrame([
    {  
        'genre': genre, 
        'count': data['count'], 
        'min_member': data['min_member'],
        'max_member': data['max_member'],
        'avg_member': round(data['total_member'] / data['count']),
        'min_favorite': data['min_favorite'],
        'max_favorite': data['max_favorite'],
        'avg_favorite_member': round(data['total_favorite'] / data['count']),
        
    }
    for genre, data in genre_stats.items()
])

df_genre_stats = df_genre_stats.sort_values(by='count', ascending=False).reset_index(drop=True)

df_genre_stats.style.set_properties(**{
    'background-color': 'darkgreen', 
    'color': 'white', 
    'font-weight': 'bold', 
    'border-bottom': '1px solid white'
})

In [ ]:
plt.figure(figsize=(12, 8))
plt.barh(df_genre_stats['genre'][:25], df_genre_stats['count'][:25], color=np.random.rand(128, 3)[0])
plt.xlabel('Number of Genres')
plt.ylabel('Genre')
plt.title(f'Top Genres from top {num_} animes')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.savefig(f'image/top_{num_}_genres.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
df_count = []
for i in df_genre_stats['genre'].unique():
    df_filtered = df[df['genre'].apply(lambda x: i in x)]
    df_filtered.reset_index()
    
    total = len(df_filtered)
    
    # Inisialisasi nilai default untuk 1st, 2nd, dan 3rd place
    first_place = df_filtered.iloc[0]['title'] if total >= 1 else '-'
    second_place = df_filtered.iloc[1]['title'] if total >= 2 else '-'
    third_place = df_filtered.iloc[2]['title'] if total >= 3 else '-'
    # forth_place = df_filtered.iloc[3]['title'] if total >= 3 else '-'
    # fifth_place = df_filtered.iloc[4]['title'] if total >= 3 else '-'
    
    # Tambahkan data ke df_count
    df_count.append({
        "genre": i,
        "total": total,
        "1st Place": first_place,
        "2nd Place": second_place,
        "3rd Place": third_place,
        # "4th Place": forth_place,
        # "5th Place": fifth_place,
    })

df_count = pd.DataFrame(df_count)
df_count.style.set_properties(**{
    'background-color': 'darkgreen', 
    'color': 'white', 
    'font-weight': 'bold', 
    'border-bottom': '1px solid white'
})

In [ ]:
print(df_genre_stats['genre'].unique())
len(df_genre_stats['genre'].unique())

# Analysis time improvement per Genre

In [ ]:
Genre = 'Romance'
df_filtered = df[df['genre'].apply(lambda x: Genre in x)]
print(f'Total {Genre} Animes: {len(df_filtered)}')

In [ ]:
genre_pure = df[df['genre'].apply(lambda x: x == [Genre])]

genre_mixed = df[df['genre'].apply(lambda x: Genre in x and len(x) > 1)]

In [ ]:
genre_pure.head(3)

In [ ]:
genre_mixed.head(3)

In [ ]:
len(genre_pure)

In [ ]:
len(genre_mixed)

In [ ]:
# # Rata-rata member
pure_member = genre_pure['member'].mean()
pure_member_min = genre_pure['member'].min()
pure_member_max = genre_pure['member'].max()


mixed_member = genre_mixed['member'].mean()
mixed_member_min = genre_mixed['member'].min()
mixed_member_max = genre_mixed['member'].max()

print('=' * 50)
print(f'Rata-rata Member {Genre} Murni: {format_number(pure_member)}')
print(f'Nilai Member Terendah {Genre} Murni: {format_number(pure_member_min)}')
print(f'Nilai Member Tertinggi {Genre} Murni: {format_number(pure_member_max)}')
print('=' * 50)
print(' ' * 50)
print('=' * 50)
print(f'Rata-rata Member {Genre} Campuran: {format_number(mixed_member)}')
print(f'Nilai Member Terendah {Genre} Campuran: {format_number(mixed_member_min)}')
print(f'Nilai Member Tertinggi  {Genre} Campuran: {format_number(mixed_member_max)}')
print('=' * 50)

In [ ]:
# # Rata-rata favorite
pure_favorite = genre_pure['favorite'].mean()
pure_favorite_min = genre_pure['favorite'].min()
pure_favorite_max = genre_pure['favorite'].max()


mixed_favorite = genre_mixed['favorite'].mean()
mixed_favorite_min = genre_mixed['favorite'].min()
mixed_favorite_max = genre_mixed['favorite'].max()

print('=' * 50)
print(f'Rata-rata Favorite {Genre} Murni: {format_number(pure_favorite)}')
print(f'Nilai Favorite Terendah {Genre} Murni: {format_number(pure_favorite_min)}')
print(f'Nilai Favorite Tertinggi {Genre} Murni: {format_number(pure_favorite_max)}')
print('=' * 50)
print(' ' * 50)
print('=' * 50)
print(f'Rata-rata Favorite {Genre} Campuran: {format_number(mixed_favorite)}')
print(f'Nilai Favorite Terendah {Genre} Campuran: {format_number(mixed_favorite_min)}')
print(f'Nilai Favorite Tertinggi  {Genre} Campuran: {format_number(mixed_favorite_max)}')
print('=' * 50)

In [ ]:
co_genres = []
for genres in df_filtered['genre']:
    co_genres.extend([g for g in genres if g != Genre])

genre_counts = Counter(co_genres)

genre_df = pd.DataFrame(genre_counts.items(), columns=['Genre', 'Count'])
genre_df = genre_df.sort_values(by='Count', ascending=False)

plt.figure(figsize=(12, 6))
bars = plt.bar(genre_df['Genre'][:10], genre_df['Count'][:10], color='darkgreen')
plt.title(f'Genres that Co-occur with "{Genre}"', fontfamily='Times New Roman')
plt.xlabel('Genre', fontfamily='Times New Roman')
plt.ylabel('Count', fontfamily='Times New Roman')
plt.xticks(rotation=90, fontfamily='Times New Roman', fontsize=12)

for bar in bars:
    yval = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width() / 2,
        yval * 0.5,
        f'{int(yval):,}'.replace(',', '.'),
        ha='center',
        va='center',
        color='white',
        fontsize=24,
        fontfamily='Times New Roman', 
        fontweight='bold',
        rotation=90
    )
        
plt.tight_layout()
plt.show()

In [ ]:
monthly_stats = genre_analysis(Genre, df, 10)

In [ ]:
decade_stats = genre_analysis_decade(Genre, df, 1)
# print(decade_stats.to_markdown())

# Forecast Trend Genre

In [ ]:
heatmap_data = plot_genre_heatmap(df) # top_n_genres=(len(df_genre_stats['genre'].unique())) , genres=["Slice of Life", "Action", "Isekai"]

In [ ]:
# Run the analysis
trend_analysis = analyze_genre_trends(heatmap_data)
trend_analysis.head(15)

In [ ]:
# Visualize the results
visualize_genre_trends(heatmap_data, trend_analysis, top_n=15)

In [ ]:
(len(df_genre_stats['genre'].unique()))

In [ ]:
# Generate and print the analysis
trend_report = generate_trend_analysis(trend_analysis)
print(trend_report)